In [80]:
from pyspark.sql import  SparkSession
spark = SparkSession.builder.master("local[*]")\
.appName("stockprice").config("spark.excuter.memory","6gb").getOrCreate()
sc = spark.sparkContext

In [81]:
from pyspark.ml.classification import DecisionTreeClassificationModel
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml import Pipeline,PipelineModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.linalg import Vector,Vectors
from pyspark.sql import Row
from pyspark.ml.feature import IndexToString,StringIndexer,VectorIndexer


In [82]:
def  f(x):
    rel = {}
    rel['fearures'] = Vectors.dense(float(x[0]),float(x[1]),float(x[2]),float(x[3]))
    rel['label'] = str(x[4])
    return rel

In [83]:
data = spark.sparkContext.textFile('iristree.txt').\
    map(lambda line: line.split(',')).\
    map(lambda p:Row(**f(p))).\
    toDF()

In [84]:
data.show(5)

+-----------------+-----------+
|         fearures|      label|
+-----------------+-----------+
|[5.1,3.5,1.4,0.2]|Iris-setosa|
|[4.9,3.0,1.4,0.2]|Iris-setosa|
|[4.7,3.2,1.3,0.2]|Iris-setosa|
|[4.6,3.1,1.5,0.2]|Iris-setosa|
|[5.0,3.6,1.4,0.2]|Iris-setosa|
+-----------------+-----------+
only showing top 5 rows



In [85]:
labelIndex = StringIndexer(inputCol='label',outputCol='indexlabel').fit(data)

In [86]:
featureIndex = VectorIndexer(inputCol='fearures',outputCol='indexfeatures',maxCategories=4).fit(data)

In [87]:
laberCoventer = IndexToString(inputCol='indexlabel',outputCol='predictionlabel',labels=labelIndex.labels)

In [88]:
training,testing = data.randomSplit([0.7,0.3],seed = 2020)

In [89]:
dfClassifer = DecisionTreeClassifier(featuresCol='indexfeatures',labelCol='indexlabel',
                                    )

In [90]:
Piple = Pipeline(stages=[labelIndex,featureIndex,laberCoventer,dfClassifer])

In [91]:
model = Piple.fit(training)

In [92]:
dfPrediction = model.transform(testing)

In [93]:
dfPrediction.show(5)

+-----------------+---------------+----------+-----------------+---------------+--------------+-------------+----------+
|         fearures|          label|indexlabel|    indexfeatures|predictionlabel| rawPrediction|  probability|prediction|
+-----------------+---------------+----------+-----------------+---------------+--------------+-------------+----------+
|[4.6,3.6,1.0,0.2]|    Iris-setosa|       0.0|[4.6,3.6,1.0,0.2]|    Iris-setosa|[39.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|[4.8,3.0,1.4,0.3]|    Iris-setosa|       0.0|[4.8,3.0,1.4,0.3]|    Iris-setosa|[39.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|[4.8,3.1,1.6,0.2]|    Iris-setosa|       0.0|[4.8,3.1,1.6,0.2]|    Iris-setosa|[39.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|[4.8,3.4,1.9,0.2]|    Iris-setosa|       0.0|[4.8,3.4,1.9,0.2]|    Iris-setosa|[39.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|[4.9,2.4,3.3,1.0]|Iris-versicolor|       1.0|[4.9,2.4,3.3,1.0]|Iris-versicolor|[0.0,35.0,0.0]|[0.0,1.0,0.0]|       1.0|
+-----------------+-------------

In [94]:
dfPrediction.select('predictionlabel','label','fearures').show()

+---------------+---------------+-----------------+
|predictionlabel|          label|         fearures|
+---------------+---------------+-----------------+
|    Iris-setosa|    Iris-setosa|[4.6,3.6,1.0,0.2]|
|    Iris-setosa|    Iris-setosa|[4.8,3.0,1.4,0.3]|
|    Iris-setosa|    Iris-setosa|[4.8,3.1,1.6,0.2]|
|    Iris-setosa|    Iris-setosa|[4.8,3.4,1.9,0.2]|
|Iris-versicolor|Iris-versicolor|[4.9,2.4,3.3,1.0]|
|    Iris-setosa|    Iris-setosa|[4.9,3.1,1.5,0.1]|
|    Iris-setosa|    Iris-setosa|[4.9,3.1,1.5,0.1]|
|    Iris-setosa|    Iris-setosa|[5.0,3.0,1.6,0.2]|
|    Iris-setosa|    Iris-setosa|[5.1,3.3,1.7,0.5]|
|    Iris-setosa|    Iris-setosa|[5.1,3.8,1.5,0.3]|
|    Iris-setosa|    Iris-setosa|[5.1,3.8,1.9,0.4]|
|    Iris-setosa|    Iris-setosa|[5.7,4.4,1.5,0.4]|
|Iris-versicolor|Iris-versicolor|[6.1,2.8,4.0,1.3]|
|Iris-versicolor|Iris-versicolor|[6.1,2.8,4.7,1.2]|
|Iris-versicolor|Iris-versicolor|[6.1,2.9,4.7,1.4]|
|Iris-versicolor|Iris-versicolor|[6.5,2.8,4.6,1.5]|
|Iris-versic

In [97]:
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction',labelCol='indexlabel') 

In [98]:
dfAccuracy = evaluator.evaluate(dfPrediction)
dfAccuracy

0.9411764705882353